## madxp package

This package aims to complement the cpymad package with some additional functions, namely to access the MAD-X variables.
In a second stage, we will propose to merge with the main package cpymad.



## To install the package 
One can install the package by
```
pip install --user git+https://github.com/sterbini/madxp.git
```
or upgrade it by
```
pip install --upgrade --user git+https://github.com/sterbini/madxp.git
```
The numpy, pandas and cpymad packages are required.

## Example
In the following we will present with simple example the main functions of the package.
It will be also an occasion to introduce the jargon we will use.

After a standard import 

In [2]:
from madxp import cpymadTool as mt
from cpymad.madx import Madx
mad=Madx()


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2020.04.19 10:26:43      +
  ++++++++++++++++++++++++++++++++++++++++++++


we setup a simple MAD-X input file

In [3]:
mad.input('''
! variables definition
a=1;
b:=c+3*a+sqrt(d)+pi;
c:=a+2+e; 
d:=2+f;
f=g;
h:=3;
const i=2;

! element definition
my_quad: quadrupole, l=1, k1:=(myk1+h+b)/1000;

! sequence definition
my_sequence: sequence,l=10, refer=exit;
q1: my_quad, at=3;
endsequence;

! beam defintion
beam, sequence=my_sequence;
''');

In the file we defined constants (e.g., **i**), independent variables (e.g., **a**), dependent variables (e.g., **b**).
To be noted that **k1** (the parameter of **my_quad** depends on **myk1**, that is not explicitly defined).

Finally, we define a sequence and a beam.

The following **sequences_df** function

In [4]:
mt.sequences_df(mad)

,beam,expanded
my_sequence,False,False


is used to extract a pandas dataframe (df) contained the sequences in the **mad** handle.
NB: there is only one  sequence and has not yet be USEd: this implies that is not expanded (not drifts/checks). 

Despite the fact the beam is defined to be attached to the sequence, no beam is attached to the sequence (one needs to USE the sequence to attach it). 

Similarly on can use the **beams_df** function to list the beams that are attached to the sequence.

In [6]:
mt.beams_df(mad)

The sequence my_sequence has no beam attached.


""


In our case the beams df is empty since no beam is attached yet (see above).

One can export the variable of the present **mad** workspace.

In [7]:
my_variable_dict=mt.variables_dict(mad)
my_variable_dict.keys()

dict_keys(['constant_df', 'independent_variable_df', 'dependent_variable_df'])

There are three df's representing MAD-X global workspace. 
- constant_df, contains the constant (system and user defined)
- independent_variable_df, contains the indipendent variables
- dependent_variable_df, contains the dependent variables

In [8]:
my_variable_dict['constant_df']

,value
amu0,1.25664e-06
clight,2.99792e+08
degrad,57.2958
e,2.71828
emass,0.000510999
erad,2.81794e-15
hbar,6.58212e-25
i,2
mumass,0.105658
nmass,0.931494


NB: in addition to the MAD-X predefined constants, **i** (user defined constant) is present too.

In [9]:
my_variable_dict['independent_variable_df']

,value
a,1
f,0
g,0
h,3
none,0
twiss_tol,1e-06



NB: 
 - **g** is not explicitly declared, then is an independent variable (with 0 values)
 - despite **h** is assigned via a deferred expression, it is an independent variables (trivial deferred expression)
 - the variable **myk1** (in the definition of the **my_quad** element) is not present in the list of the independent variable (since the sequence is not yet used) 
 - the **none** and **twiss_tol** are system-defined independent variables.

In [10]:
my_variable_dict['dependent_variable_df']


,value,expression,parameters,knobs
b,13.2741,c+3*a+sqrt(d)+pi,"[a, c, d, pi]","[a, f]"
c,5.71828,a+2+e,"[a, e]",[a]
d,2,2+f,[f],[f]



The dependent variable DF contains the numerical values, the expression, the parameters and the knobs for each dependent variables. 

The *knobs of a given dependent variable*, my_variable, are a set of  independent variables that control the value of my_variable. 

NB: 
 - for **b**, **pi** is not a knobs (since it is a constant). The same argument holds for **c**  and its parameter **e** (Euler number),
 - to determine the knobs, all parameters that are dependent variables (e.g., **c** as parameter of **b**)  are *decomposed* in their independent variables.

Another interesting method **sequence_df** (not to be be conbused with **sequences_df**) allows to transform a sequence in a df.

In [11]:
mt.sequence_df(mad, 'my_sequence')

Variable myk1 not defined! Cosidered as a knob.


,position,parent,base_type,length,parameters,knobs,align_errors,aper_offset,aper_offset value,aper_tol,...,slot_id,slot_id value,thick,thick value,tilt,tilt value,type,type value,v_pos,v_pos value
,,,,,,,,,,,,,,,,,,,,,
my_sequence$start,0.0,marker,marker,0.0,[none],[none],None,[0.0],[0.0],"[0.0, 0.0, 0.0]",...,none,0,NaN,NaN,NaN,NaN,,,0.0,0.0
q1,2.0,my_quad,quadrupole,1.0,"[b, h, myk1, none]","[a, f, h, myk1, none]",None,[0.0],[0.0],"[0.0, 0.0, 0.0]",...,none,0,False,False,0.0,0.0,,,0.0,0.0
my_sequence$end,10.0,marker,marker,0.0,[none],[none],None,[0.0],[0.0],"[0.0, 0.0, 0.0]",...,none,0,NaN,NaN,NaN,NaN,,,0.0,0.0


NB:
- for each element you have the *parameters* and the *knobs* columns. This knobs are the element knobs (very similar to the knobs of the variables mentioned above),
- you can see **myk1** has knob.

As for the variable and the element, we can naturally define the knobs of a sequence.


In [12]:
mt.knobs_df(mt.sequence_df(mad, 'my_sequence'))

,multeplicity,dependences
none,3,"[my_sequence$start, q1, my_sequence$end]"
a,1,[q1]
f,1,[q1]
h,1,[q1]
myk1,1,[q1]



For each knob you have the multeplicity, that is the number of elements are controlled by that specific knob. In the **dependences** column one have the explicit list of the elements controlled by this knob.

One can do also use the same funtion to extract of the knobs of the dependent variables.

In [15]:
mt.knobs_df(my_variable_dict['dependent_variable_df'])

,multeplicity,dependences
a,2,"[b, c]"
f,2,"[b, d]"


With a similar spirit, one can sub-select a sequence df with the elements controlled only on this knob. 

In [16]:
mt.knob_df('myk1',mt.sequence_df(mad, 'my_sequence'))

,position,parent,base_type,length,parameters,knobs,align_errors,aper_offset,aper_offset value,aper_tol,...,slot_id,slot_id value,thick,thick value,tilt,tilt value,type,type value,v_pos,v_pos value
,,,,,,,,,,,,,,,,,,,,,
q1,2.0,my_quad,quadrupole,1.0,"[b, h, myk1, none]","[a, f, h, myk1, none]",None,[0.0],[0.0],"[0.0, 0.0, 0.0]",...,none,0,False,False,0.0,0.0,,,0.0,0.0


One can do the same analysis for the dependent variables df.

In [20]:
mt.knob_df('f', my_variable_dict['dependent_variable_df'])

,value,expression,parameters,knobs
b,13.2741,c+3*a+sqrt(d)+pi,"[a, c, d, pi]","[a, f]"
d,2,2+f,[f],[f]


Let us *USE* the sequence

In [22]:
mad.input('use, sequence=my_sequence;');

The we can see that

In [23]:
mt.sequences_df(mad)

,beam,expanded
my_sequence,True,True


and

In [24]:
mt.beams_df(mad)

,particle,sequence,bunched,radiate,mass,charge,energy,pc,gamma,beta,...,circ,dtbyds,deltap,alfa,u0,qs,arad,bv,pdamp,n1min
my_sequence,positron,my_sequence,True,False,0.000511,1.0,1.0,1.0,1956.951198,1.0,...,0.0,0.0,0.0,2.611199e-07,0.0,0.0,2.817940e-15,1.0,"[1.0, 1.0, 2.0]",-1.0


To ease the visualization of a given element you can run

In [25]:
mt.show_element('q1', mt.sequence_df(mad, 'my_sequence'))

q1
position                                     2
parent                                 my_quad
base_type                           quadrupole
length                                       1
parameters                  [b, h, myk1, none]
knobs                    [a, f, h, myk1, none]
aper_offset                              [0.0]
aper_offset value                        [0.0]
aper_tol                       [0.0, 0.0, 0.0]
aper_tol value                 [0.0, 0.0, 0.0]
aperture                                 [0.0]
aperture value                           [0.0]
apertype                                circle
apertype value                          circle
assembly_id                               none
assembly_id value                            0
at                                         3.0
at value                                     3
base_name                           quadrupole
bend_fringe                              False
bend_fringe value                        False
calib     

,position,parent,base_type,length,parameters,knobs,aper_offset,aper_offset value,aper_tol,aper_tol value,...,slot_id,slot_id value,thick,thick value,tilt,tilt value,type,type value,v_pos,v_pos value
,,,,,,,,,,,,,,,,,,,,,
q1,2.0,my_quad,quadrupole,1.0,"[b, h, myk1, none]","[a, f, h, myk1, none]",[0.0],[0.0],"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",...,none,0,False,False,0.0,0.0,,,0.0,0.0


One can note that, after using the sequence, **myk1** appears in the independent variables df.

In [26]:
my_variable_dict=mt.variables_dict(mad)
my_variable_dict['independent_variable_df']

,value
a,1
f,0
g,0
h,3
myk1,0
none,0
twiss_tol,1e-06


We can simply twiss the active sequence.

In [28]:
mad.input('twiss, betx=1,bety=1;')

enter Twiss module

++++++ table: summ

            length             orbit5               alfa            gammatr 
                10                 -0                  0                  0 

                q1                dq1            betxmax              dxmax 
      0.2356243527                  0        94.80191493                  0 

             dxrms             xcomax             xcorms                 q2 
                 0                  0                  0       0.2327327674 

               dq2            betymax              dymax              dyrms 
                 0        107.4425105                  0                  0 

            ycomax             ycorms             deltap            synch_1 
                 0                  0                  0                  0 

           synch_2            synch_3            synch_4            synch_5 
                 0                  0                  0                  0 

            nflips 
          

True

and we can list the tables of the MAD-X instance by using


In [29]:
list(mad.table)

['summ', 'twiss']

We can import in df the two tables *twiss* and *summ* (the summary table) by

In [30]:
mt.table_df(mad.table.twiss)

,name,keyword,s,betx,alfx,mux,bety,alfy,muy,x,...,sig54,sig55,sig56,sig61,sig62,sig63,sig64,sig65,sig66,n1
,,,,,,,,,,,,,,,,,,,,,
my_sequence$start:1,my_sequence$start:1,marker,0.0,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
drift_0:0,drift_0:0,drift,2.0,5.000000,-2.000000,0.176208,5.000000,-2.000000,0.176208,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q1:1,q1:1,quadrupole,3.0,9.870401,-2.843951,0.198879,10.130788,-3.158591,0.198706,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
drift_1:0,drift_1:0,drift,10.0,94.801915,-9.289122,0.235624,107.442511,-10.743083,0.232733,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
my_sequence$end:1,my_sequence$end:1,marker,10.0,94.801915,-9.289122,0.235624,107.442511,-10.743083,0.232733,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


and

In [31]:
mt.summ_df(mad.table.summ)

,length,orbit5,alfa,gammatr,q1,dq1,betxmax,dxmax,dxrms,xcomax,...,dyrms,ycomax,ycorms,deltap,synch_1,synch_2,synch_3,synch_4,synch_5,nflips
summ,10.0,-0.0,0.0,0.0,0.235624,0.0,94.801915,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In needed one can "interpolate" the optical function at specific *s*-position using the function table_interpolation_df. The "interpolation" recompute with a minimal twiss the correct values (therefore is not a proper interpolation but re-computation).  

In [32]:
import numpy as np
mt.table_interpolation_df(my_s_list=np.linspace(2,3,11), my_table=mt.table_df(mad.table.twiss))

,name,keyword,s,betx,alfx,mux,bety,alfy,muy,x,...,sig54,sig55,sig56,sig61,sig62,sig63,sig64,sig65,sig66,n1
drift_0:0,drift_0:0,drift,2,5,-2,0.176208,5,-2,0.176208,0,...,0,0,0,0,0,0,0,0,0,0
at_2.1:1,at_2.1:1,interpolation,2.1,5.40914,-2.0912,0.179269,5.41086,-2.1088,0.179268,0,...,0,0,0,0,0,0,0,0,0,0
at_2.2:1,at_2.2:1,interpolation,2.2,5.83639,-2.18104,0.182101,5.84361,-2.21897,0.182099,0,...,0,0,0,0,0,0,0,0,0,0
at_2.3:1,at_2.3:1,interpolation,2.3,6.28146,-2.26946,0.18473,6.29854,-2.33059,0.184723,0,...,0,0,0,0,0,0,0,0,0,0
at_2.4:1,at_2.4:1,interpolation,2.4,6.74408,-2.35641,0.187175,6.77595,-2.44372,0.187159,0,...,0,0,0,0,0,0,0,0,0,0
at_2.5:1,at_2.5:1,interpolation,2.5,7.22393,-2.44182,0.189455,7.27614,-2.55845,0.189426,0,...,0,0,0,0,0,0,0,0,0,0
at_2.6:1,at_2.6:1,interpolation,2.6,7.7207,-2.52564,0.191587,7.79944,-2.67484,0.191538,0,...,0,0,0,0,0,0,0,0,0,0
at_2.7:1,at_2.7:1,interpolation,2.7,8.23407,-2.60782,0.193583,8.34619,-2.79297,0.193511,0,...,0,0,0,0,0,0,0,0,0,0
at_2.8:1,at_2.8:1,interpolation,2.8,8.76371,-2.68829,0.195456,8.91675,-2.91292,0.195356,0,...,0,0,0,0,0,0,0,0,0,0
at_2.9:1,at_2.9:1,interpolation,2.9,9.30927,-2.76702,0.197218,9.51149,-3.03477,0.197084,0,...,0,0,0,0,0,0,0,0,0,0


The table_interpolation_df is somehow equivalent to the *intepolate* flag of MAD-X, as you can see below

In [34]:
mad.input('''
select, flag=interpolate, clear;
select, flag=interpolate, sequence=my_sequence,class=quadrupole, slice=10;
! more on 
twiss, betx=1,bety=1,table='with_interpolation';
''')
mt.table_df(mad.table.with_interpolation)

enter Twiss module

++++++ table: summ

            length             orbit5               alfa            gammatr 
                10                 -0                  0                  0 

                q1                dq1            betxmax              dxmax 
      0.2356243527                  0        94.80191493                  0 

             dxrms             xcomax             xcorms                 q2 
                 0                  0                  0       0.2327327674 

               dq2            betymax              dymax              dyrms 
                 0        107.4425105                  0                  0 

            ycomax             ycorms             deltap            synch_1 
                 0                  0                  0                  0 

           synch_2            synch_3            synch_4            synch_5 
                 0                  0                  0                  0 

            nflips 
          

,name,keyword,s,betx,alfx,mux,bety,alfy,muy,x,...,sig54,sig55,sig56,sig61,sig62,sig63,sig64,sig65,sig66,n1
,,,,,,,,,,,,,,,,,,,,,
my_sequence$start:1,my_sequence$start:1,marker,0.0,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
drift_0:0,drift_0:0,drift,2.0,5.000000,-2.000000,0.176208,5.000000,-2.000000,0.176208,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q1:1,q1:1,quadrupole,2.1,5.409142,-2.091202,0.179269,5.410858,-2.108800,0.179268,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q1:1,q1:1,quadrupole,2.2,5.836390,-2.181043,0.182101,5.843611,-2.218972,0.182099,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q1:1,q1:1,quadrupole,2.3,6.281465,-2.269464,0.184730,6.298543,-2.330590,0.184723,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q1:1,q1:1,quadrupole,2.4,6.744077,-2.356408,0.187175,6.775948,-2.443724,0.187159,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q1:1,q1:1,quadrupole,2.5,7.223926,-2.441818,0.189455,7.276139,-2.558449,0.189426,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q1:1,q1:1,quadrupole,2.6,7.720698,-2.525639,0.191587,7.799439,-2.674840,0.191538,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q1:1,q1:1,quadrupole,2.7,8.234072,-2.607815,0.193583,8.346191,-2.792972,0.193511,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


but the proposed function can do the interpolation without a "live" MAD-X handle and sequence.